In [26]:
%pwd

'/Users/spartan/Desktop/NLP/Text_Summerizer/Text-Summarizer'

In [27]:
import os
# os.chdir("../")

In [28]:
%pwd

'/Users/spartan/Desktop/NLP/Text_Summerizer/Text-Summarizer'

In [29]:
import sys
import os

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "src")))

In [30]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path
    

In [31]:
from textSummarizer.constant import *
from textSummarizer.utils.common import read_yaml,create_directories

In [32]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        param_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(param_filepath)
        
        create_directories([self.config.artifacts_root])

    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
    
        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )

        return data_ingestion_config


In [33]:
import os
import urllib.request as request
import zipfile
from textSummarizer.logging import logger
from textSummarizer.utils.common import get_size


In [34]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename= self.config.local_data_file
            )

            logger.info(f"{filename} downloaded! with following info: \n{headers}")

        else:
            logger.info(f"file already exists of size: {get_size(Path(self.config.local_data_file))}")

        
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """

        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok= True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [35]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2025-03-17 01:12:08,268: INFO: common : yaml file: config/config.yaml loaded successfully]
[2025-03-17 01:12:08,272: INFO: common : yaml file: params.yaml loaded successfully]
[2025-03-17 01:12:08,274: INFO: common : created dictionary at: artifacts]
[2025-03-17 01:12:08,275: INFO: common : created dictionary at: artifacts/data_ingestion]
[2025-03-17 01:12:14,849: INFO: 1484994986 : artifacts/data_ingestion/data.zip downloaded! with following info: 
Connection: close
Content-Length: 7903594
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "dbc016a060da18070593b83afff580c9b300f0b6ea4147a7988433e04df246ca"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: AB50:1132:262A310:307E47C:67D7D882
Accept-Ranges: bytes
Date: Mon, 17 Mar 2025 08:08:35 GMT
Via: 1.1 varnish
X-Served-By: cache-pao-kpao177006